# Simon's Algorithm
### Authors: Shane Barratt, Yash Shah, Julien Bloch

## Introduction

You've probably heard of Quantum Computing, either in the news or from your friends. But what's all the hype? Can a quantum computer really outperform a classical computer? Simon says it can.

In computability theory, a Turing Machine, introduced by Alan Turing, is a theoretical machine used to examine the possibilities and limitations of computers. All modern silicon-based computers are "Turing-complete", in the sense that they can perform any computation that a Turing Machine can. The theory is further extended to something called a Probabilstic Turing Machine, a non-deterministic Turing Machine which chooses between state transitions based on a probability distribution.

Simon, in his seminal 1994 paper "On the Power of Quantum Computation, introduced a problem and quantum algorithm which provides compelling evidence for a computational advantage of the quantum model of computation over probabilistic Turing machines.

This Jupyter Notebook seeks to provide a simulation of his algorithm, as well as interactive widgets and plots for you to test out different parameters.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random
%matplotlib inline

//anaconda/envs/tensorflow/lib/python3.5/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
//anaconda/envs/tensorflow/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
//anaconda/envs/tensorflow/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


The basic setup of Simon's algorithm is that you're given a function $f: \{ 0,1 \}^n \to \{ 0,1 \}^m$, with $m \geq n$ and either 

a) f is 1-to-1   
or   
b) there exists an $s$ such that $\forall x \neq x'$, $f(x) = f(x') \iff x' = x \oplus s$.




We want to determine which of these holds for $f$ and find $s$ in the second case. This problem seems impossible from the classical point of view; it will require enumeration of all possible $s$'s which would take time exponential in $n$.

We begin by defining multiple functions for use later.

In [3]:
def num_to_binary(x, n):
    # Returns x in n-bit binary representation
    st = ("{0:0%db}" % n).format(x)
    return np.array([int(s) for s in st])

def bitwise_dot_product(x, y, n):
    # Returns x.y (mod 2), where x and y are both n bits
    return np.sum(num_to_binary(np.bitwise_and(x, y), n)) % 2

def random_one_to_one_function(n, m):
    # Returns a random 1-1 f: {0,1}^n -> {0,1}^m
    assert m >= n
    xs = list(range(2**n))
    ys = list(range(2**m))
    random.shuffle(ys)
        
    def f(x):
        return ys[x]
    
    return f

def random_s_function(n, m, s):
    # Returns a random f: {0,1}^n -> {0,1}^m s.t. 
    assert m >= n
    assert s != 0
    
    xs = list(range(2**n))
    ys = list(range(2**m))
    random.shuffle(ys)
    
    for x in xs:
        ys[np.bitwise_and(x, s)] = ys[x]
    
    def f(x):
        return ys[x]
    
    return f

Simon proposes an algorithm "Fourier-Twice" to solve this very problem. It is repeated here for your convenience:

Routine **Fourier-twice**
1. perform a Hadamard transformation on a string of $n$ zeros, producing $2^{-n/2}\sum_x |x>$
2. compute $f(x)$, concatenating the answer to $x$, thus producing $2^{-n/2} \sum_x |(x, f(x))>$
3. perform a Hadamard transformation on $x$, producing $2^{-n} \sum_y \sum_x (-1)^{x . y} |(y, f(x))>$

End **Fourier-twice**

The entire algorithm is to run **Fourier-twice** $n-1$ times, each time sampling the final superposition and noting the resulting $(y, f(x))$. Simon's algorithm is depicted pictorally below.

<img src="simons-diagram.png" alt="Drawing" style="width: 400px;"/>

In case a, where f is 1-1, the sum after step 3 of the algorithm will be a uniform superposition across all $(y, f(x))$ pairs. The algorithm will return $n-1$ i.i.d. $(y, f(x))$ pairs uniformly distributed amongst n-bit strings.

In case b, where f follows the structure described above (there exists an $s$ such that $\forall x \neq x'$, $f(x) = f(x') \iff x' = x \oplus s$), the sum after step 3 of the algorithm will be a uniform superposition across all $(y, f(x))$ s.t. $y.s=0$ because all terms where $y.s=1$ cancel out.

In [6]:
def fourier_twice(f, n, s, onetoone=False):
    x = random.randint(0, n-1)
    y = random.randint(0, n-1)
    if not onetoone:
        possible_ys = []
        for y in range(2**n):
            if bitwise_dot_product(y, s, n) == 0:
                possible_ys.append(y)
        y = random.choice(possible_ys)
    return y, f(x)

In [22]:
s = 0b1011011011
n = 10
m = 10

f = random_s_function(n, m, s)

outputs = [fourier_twice(f, n, s) for _ in range(n-1)]

print (s)

731


In [23]:
print (outputs)

[(371, 146), (76, 86), (920, 901), (242, 487), (564, 901), (756, 146), (520, 791), (241, 86), (227, 487)]


In [24]:
# print solutions
for possible_s in range(2**n):
    good = True
    for y, f_x in outputs:
        good = good and (np.sum(num_to_binary(np.bitwise_and(y, possible_s), n)) % 2 == 0)
    if good:
        print (possible_s)

0
215
524
731


In [27]:
s = 0b1011011011
n = 10
m = 10

f = random_one_to_one_function(n, m)

# how to collect n-1 independent
outputs = [fourier_twice(f, n, s, True) for _ in range(n)]

In [28]:
# print solutions
for possible_s in range(2**n):
    good = True
    for y, f_x in outputs:
        good = good and (np.sum(num_to_binary(np.bitwise_and(y, possible_s), n)) % 2 == 0)
    if good:
        print (possible_s)

0
16
32
48
64
80
96
112
128
144
160
176
192
208
224
240
256
272
288
304
320
336
352
368
384
400
416
432
448
464
480
496
512
528
544
560
576
592
608
624
640
656
672
688
704
720
736
752
768
784
800
816
832
848
864
880
896
912
928
944
960
976
992
1008
